In [1]:
games_data = []  # Assicurati che la lista sia inizializzata

import time
import requests
import xml.etree.ElementTree as ET

In [2]:
games_data = []  # Assicurati che la lista sia inizializzata

def get_data_with_backoff(game_id, max_retries=5):
    url = f"https://boardgamegeek.com/xmlapi2/thing?id={game_id}&stats=1"
    delay = 2  # Ritardo iniziale in secondi
    for i in range(max_retries):
        try:
            response = requests.get(url)
            response.raise_for_status()
            time.sleep(1)  # Pausa per evitare di sovraccaricare l'API

            # Parsing XML
            root = ET.fromstring(response.content)
            item = root.find("item")
            
            # Verifica presenza di dati
            if item is not None:
                # Usa il metodo `find` in modo sicuro con un controllo per ogni elemento
                year_published = item.find("yearpublished")
                year_published = int(year_published.get("value", 0)) if year_published is not None else None
                
                # Filtra solo i giochi dal 2022 in poi
                if year_published and year_published >= 2022:
                    game_info = {
                        "BGGId": game_id,
                        "Name": item.find("name").get("value") if item.find("name") is not None else "N/A",
                        "YearPublished": year_published,
                        "GameWeight": float(item.find("statistics/ratings/averageweight").get("value", 0)) if item.find("statistics/ratings/averageweight") is not None else 0,
                        "AvgRating": float(item.find("statistics/ratings/average").get("value", 0)) if item.find("statistics/ratings/average") is not None else 0,
                        "BayesAvgRating": float(item.find("statistics/ratings/bayesaverage").get("value", 0)) if item.find("statistics/ratings/bayesaverage") is not None else 0,
                        "NumOwned": int(item.find("statistics/ratings/owned").get("value", 0)) if item.find("statistics/ratings/owned") is not None else 0,
                        "NumComments": int(item.find("statistics/ratings/numcomments").get("value", 0)) if item.find("statistics/ratings/numcomments") is not None else 0,
                        "NumUserRatings": int(item.find("statistics/ratings/usersrated").get("value", 0)) if item.find("statistics/ratings/usersrated") is not None else 0,
                        "MinPlayers": int(item.find("minplayers").get("value", 1)) if item.find("minplayers") is not None else 1,
                        "MaxPlayers": int(item.find("maxplayers").get("value", 1)) if item.find("maxplayers") is not None else 1,
                        "MinPlaytime": int(item.find("minplaytime").get("value", 0)) if item.find("minplaytime") is not None else 0,
                        "MaxPlaytime": int(item.find("maxplaytime").get("value", 0)) if item.find("maxplaytime") is not None else 0,
                        "AgeRec": int(item.find("minage").get("value", 0)) if item.find("minage") is not None else 0,
                        "Description": item.find("description").text.strip() if item.find("description") is not None else ""
                    }
                    games_data.append(game_info)
                    print(f"Recuperati dati per il gioco {game_id}")
                    return game_info  # Esce dalla funzione se il recupero ha successo

        except requests.exceptions.HTTPError as e:
            if response.status_code == 429:
                print(f"Errore 429: Tentativo {i + 1} - Attendo {delay} secondi prima di riprovare...")
                time.sleep(delay)
                delay *= 2  # Raddoppia il ritardo
            else:
                print(f"Errore nel recuperare i dati per il gioco {game_id}: {e}")
                break
        except Exception as e:
            print(f"Errore generico per il gioco {game_id}: {e}")
            break

    return None  # Ritorna None se non riesce dopo max_retries tentativi

In [ ]:
# Esegui per ogni ID di gioco
game_ids = range(359037,400000)  # La lista di ID di giochi che vuoi recuperare
for game_id in game_ids:
    get_data_with_backoff(game_id)


Recuperati dati per il gioco 359037
Recuperati dati per il gioco 359040
Recuperati dati per il gioco 359041
Recuperati dati per il gioco 359042
Recuperati dati per il gioco 359043
Recuperati dati per il gioco 359044
Recuperati dati per il gioco 359046
Recuperati dati per il gioco 359048
Recuperati dati per il gioco 359049
Recuperati dati per il gioco 359050
Recuperati dati per il gioco 359051
Recuperati dati per il gioco 359052
Recuperati dati per il gioco 359053
Recuperati dati per il gioco 359055
Recuperati dati per il gioco 359056
Recuperati dati per il gioco 359062
Recuperati dati per il gioco 359065
Recuperati dati per il gioco 359066
Recuperati dati per il gioco 359069
Recuperati dati per il gioco 359073
Recuperati dati per il gioco 359074
Recuperati dati per il gioco 359078
Recuperati dati per il gioco 359080
Recuperati dati per il gioco 359082
Recuperati dati per il gioco 359087
Recuperati dati per il gioco 359088
Recuperati dati per il gioco 359089
Recuperati dati per il gioco

In [ ]:

# Stampa i dati raccolti o esegui altre operazioni
print(f"Totale giochi recuperati: {len(games_data)}")


In [ ]:
import pandas as pd
# Creazione del DataFrame
games_df = pd.DataFrame(games_data)

# Salva i dati in un file CSV
games_df.to_csv("games_2022daggioa_to_present.csv", index=False)
print("Dati salvati in games_2022daggioa_to_present.csv")

Dati salvati in games_2022_to_present.csv
